In [1]:
!pip install langchain langchain-openai langchain-community faiss-cpu tiktoken

^C


In [3]:
import os
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

In [4]:
key = 

In [34]:
os.environ["OPENAI_API_KEY"] = key

In [11]:
loader = TextLoader('srk.txt', encoding='utf-8')

In [12]:
documents = loader.load()

In [14]:
print(documents)

[Document(metadata={'source': 'srk.txt'}, page_content='Shah Rukh Khan[a] (pronounced [ˈʃɑːɦɾʊx xɑːn] ⓘ; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi cinema. Referred to in the media as the "Baadshah of Bollywood" and "King Khan",[b] he has appeared in more than 100 films, and earned numerous accolades, including 14 Filmfare Awards. He has been awarded the Padma Shri by the Government of India, as well as the Order of Arts and Letters and Legion of Honour by the Government of France. Khan has a significant following in Asia and the Indian diaspora worldwide. In terms of audience size and income, several media outlets have described him as one of the most successful film stars in the world.[c] Many of his films thematise Indian national identity and connections with diaspora communities, or gender, racial, social and religious differences and grievances.\n\nKhan began his career with appearances in several television se

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len
)

In [30]:
docs = text_splitter.split_documents(documents)

In [31]:
len(docs)

160

In [32]:
docs[40]

Document(metadata={'source': 'srk.txt'}, page_content="moral dilemma. India Today cites it as one of the first major Bollywood pictures to succeed in the United States.[75] Khan's final release of 1997 was the second collaboration with Yash Chopra in the popular musical romance Dil To Pagal Hai. He portrayed Rahul, a stage director caught in a love triangle between Madhuri Dixit and Karisma Kapoor. The film and his performance met with critical praise, winning Khan his third Best Actor Award at Filmfare.[58]")

In [35]:
embedding = OpenAIEmbeddings()

In [36]:
library = FAISS.from_documents(documents=docs, embedding=embedding)

In [37]:
Q1 = "Who is SRK's wife?"
Q2 = "How old is SRK?"
Q3 = "Is srk a millionaire?"

In [38]:
Query_Answer = library.similarity_search(Q1,k=3)

In [46]:
Query_Answer[0].page_content

'Personal life\nShah Rukh Khan standing beside his wife Gauri at a party in 2012\nKhan with his wife Gauri in 2012'

In [49]:
docs_and_scores = library.similarity_search_with_score(Q2)

In [53]:
docs_and_scores

[(Document(id='4ed50afc-ac05-4909-9130-0905287b1ec3', metadata={'source': 'srk.txt'}, page_content='Shah Rukh Khan[a] (pronounced [ˈʃɑːɦɾʊx xɑːn] ⓘ; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi cinema. Referred to in the media as the "Baadshah of Bollywood" and "King Khan",[b] he has appeared in more than 100 films, and earned numerous accolades, including 14 Filmfare Awards. He has been awarded the Padma Shri by the Government of India, as well as the Order of Arts and Letters and Legion of Honour by the Government of France.'),
  0.2927246),
 (Document(id='f5be8504-e320-466b-8f59-04c8fc312875', metadata={'source': 'srk.txt'}, page_content='As of 2015, Khan is co-chairman of the motion picture production company Red Chillies Entertainment and its subsidiaries and is the co-owner of the Indian Premier League cricket team Kolkata Knight Riders and the Caribbean Premier League team Trinbago Knight Riders. The media often

In [61]:
retriever = library.as_retriever()

In [66]:
qa = RetrievalQA.from_chain_type(
llm = OpenAI(),
    chain_type = "map_reduce",
    retriever = retriever
)

In [76]:
retriever_query = "Who is his wife?"


In [77]:
results = qa.invoke(retriever_query)

In [78]:
print(results)

{'query': 'Who is his wife?', 'result': ' His wife is not mentioned in this portion of the document.'}


In [79]:
library.save_local("faiss_index_srk")

In [80]:
srk_saved = FAISS.load_local("faiss_index_srk",embeddings=embedding)

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).